In [42]:
from bertopic import BERTopic
import tweepy
import pandas as pd
import numpy as np
import csv
import re
pd.set_option("display.max_colwidth", None)
from collections import Counter
import scipy.sparse as sp
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import itertools
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
#path=r'C:\Users\ddl11\Desktop\project\tweets'   #load data
#dem1 = pd.read_csv('dems_tweets.csv')
#dem1 = dem1[['user_id','Time','Text']]
#dem2 = pd.read_csv('dems_tweets_2.csv')
#dem2 = dem2[['user_id','Time','Text']]
#dems = pd.concat([dem1,dem2])
#dems = dems.reset_index()
#dems = dems[['user_id','Time','Text']]

rep1 = pd.read_csv('reps_tweets.csv')
rep1 = rep1[['user_id','Time','Text']]
rep2 = pd.read_csv('reps_tweets_2.csv')
rep2 = rep2[['user_id','Time','Text']]
reps = pd.concat([rep1,rep2])
reps = reps.reset_index()
reps = reps[['user_id','Time','Text']]

In [44]:
#dems['Time'] = pd.to_datetime(dems['Time'])
#dems = dems[(dems['Time'] > '2020-11-01') & (dems['Time'] < '2021-07-27')]
reps['Time'] = pd.to_datetime(reps['Time'])
reps = reps[(reps['Time'] > '2020-10-31') & (reps['Time'] < '2021-07-27')]

In [45]:
# first preprecessing
#dems['Text'] = dems['Text'].apply(lambda x:x[2:])  #edit raw text
reps['Text'] = reps['Text'].apply(lambda x:x[2:])
#dems['Text'] = dems['Text'].apply(lambda x:x[:-1])
reps['Text'] = reps['Text'].apply(lambda x:x[:-1])

#dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x99', '\'') #replace some unicodes
#dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x9c', '\"')
#dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x9d', '\"')
reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x99', '\'')
reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x9c', '\"')
reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x9d', '\"')

#dems['Text'] = dems['Text'].str.replace(r'\\x[A-Za-z0-9./]+', '')
reps['Text'] = reps['Text'].str.replace(r'\\x[A-Za-z0-9./]+', '')#remove all other unicodes

#dems['Text'] = dems['Text'].str.replace(r'\\n', ' ')
reps['Text'] = reps['Text'].str.replace(r'\\n', ' ')

In [46]:
#regular expressions
#dems['Text'] = dems.Text.str.lower()
#dems['Text'] = dems.Text.apply(lambda x: re.sub(r'\b(lol)\b', 'happyface', x))
#dems['Text'] = dems.Text.apply(lambda x: re.sub(r'https?:\/\/\S+', ' ', x))
#dems['Text'] = dems.Text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', x))
#dems['Text']= dems.Text.apply(lambda x: re.sub("@[A-Za-z0-9_]+", ' ', x))
#dems['Text']= dems.Text.apply(lambda x: re.sub("#[A-Za-z0-9_]+", ' ', x))
#dems['Text'] = dems.Text.apply(lambda x: re.sub(r"\b[0-9]+\b", ' ', x))
#dems['Text'] = dems.Text.apply(lambda x: re.sub(r"[^A-Za-z0-9 ]",'', x))
#dems['Text']= dems.Text.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", ' ', x))
#dems['Text'] = dems.Text.apply(lambda x: re.sub(r"\b[a-z0-9]\b",' ', x))
#dems['Text'] = dems.Text.apply(lambda x: re.sub(' +', ' ', x))

reps['Text'] = reps.Text.str.lower()
reps['Text'] = reps.Text.apply(lambda x: re.sub(r'\b(lol)\b', 'happyface', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r'https?:\/\/\S+', ' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', x))
reps['Text']= reps.Text.apply(lambda x: re.sub("@[A-Za-z0-9_]+", ' ', x))
reps['Text']= reps.Text.apply(lambda x: re.sub("#[A-Za-z0-9_]+", ' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r"\b[0-9]+\b", ' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r"[^A-Za-z0-9 ]",'', x))
reps['Text']= reps.Text.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", ' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r"\b[a-z0-9]\b",' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(' +', ' ', x))


In [47]:
reps = reps[reps["Text"] != ' '];reps = reps[reps["Text"] != '']
#dems = dems[dems["Text"] != ' '];dems = dems[dems["Text"] != '']
reps.index = range(len(reps));
#dems.index = range(len(dems))

In [48]:
#dems = dems.groupby('Time').sample(frac=.776) #keep dems reps the same size

In [32]:
#dems = dems.sample(660000,replace=True)
reps = reps.sample(660000,replace=True)
#dems.index = range(len(dems))
reps.index = range(len(reps))

In [33]:
docs = reps['Text']
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english", min_df=10)
model = BERTopic(vectorizer_model=vectorizer_model,nr_topics="auto",low_memory=True,top_n_words=20,
                 min_topic_size=300,verbose=True)
topics, probabilities = model.fit_transform(docs)

2021-08-06 18:53:10,785 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2021-08-06 19:59:53,810 - BERTopic - Reduced dimensionality with UMAP
2021-08-06 20:05:01,094 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-08-06 20:06:34,338 - BERTopic - Reduced number of topics from 223 to 131


In [65]:
model.get_topics()

{-1: [('election', 0.003654242457590847),
  ('democrats', 0.003466809735884624),
  ('state', 0.003455802581870456),
  ('breaking', 0.003238762698061354),
  ('vote', 0.0031525356352828477),
  ('house', 0.003099726765531821),
  ('today', 0.0029233362669784054),
  ('states', 0.002915866448623059),
  ('votes', 0.0029061363480799014),
  ('want', 0.0028226150410734607),
  ('senate', 0.002793380119607542),
  ('county', 0.002786163292629306),
  ('texas', 0.0027623796704153943),
  ('georgia', 0.002750491278663766),
  ('time', 0.0027414798590649457),
  ('make', 0.0027412037992937915),
  ('day', 0.0027202424880077673),
  ('vaccine', 0.00270626441797609),
  ('media', 0.0026782830820076954),
  ('america', 0.0026764490043921516)],
 0: [('president trump', 0.018524495812157815),
  ('donald', 0.017119075386261923),
  ('donald trump', 0.016505891825479943),
  ('president', 0.01393423178572612),
  ('trumps', 0.01341234808520116),
  ('america', 0.009717719321225077),
  ('th president', 0.0080856782266568

In [64]:
#model.get_topic_freq().values.tolist()

In [ ]:
#model.save('reps_topic_model')

In [49]:
reps['Topic'] = 0

In [56]:
#reload data, run load code above, now len reps = 1000000+
reps['Topic'][600000:] = reps['Text'][600000:].apply(lambda x: model.find_topics(x)[0][0])

In [58]:
def deal_str(data):
    data = str(data)+'\t'
    return data
reps['user_id'] = reps['user_id'].map(deal_str)

reps.to_csv('reps_with_topics.csv')